In [4]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 9.1 MB/s eta 0:00:00


In [9]:
import requests
from bs4 import BeautifulSoup

def retrieve_text_from_url(url):
    """Remove HTML tags from a string"""
    try:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")
        return soup.get_text()
    except:
        return ""


def get_news(keyboard):
    newsapi_key =''
    endpoint = 'https://newsapi.org/v2/everything'
    parameters = {
        'q' : 'tesla',
        'apiKey' : newsapi_key,
        'pageSize' : 100
    }
    data = requests.get (endpoint, params = parameters).json()
    for article in data['articles']:
        article['full_text'] = retrieve_text_from_url(article['url'])
    return data['articles']

tesla_articlse = get_news('tesla')

In [10]:
tesla_articlse = get_news('tesla')

tesla_articlse[0].keys()

dict_keys(['source', 'author', 'title', 'description', 'url', 'urlToImage', 'publishedAt', 'content', 'full_text'])

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

In [6]:
db_name = "public"
table_name = 'poh2005_tesla_articles'


# Create a table

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} (
                      source varchar(255),
                      author varchar(255),
                      title varchar(255),
                      description varchar(255),
                      url varchar(255),
                      urlToImage varchar(255),
                      publishedAt varchar(255),
                      content varchar(255),
                      full_text text)"""

with engine.connect() as connection:
  connection.execute(text(create_table_query))


In [34]:
# SQL template for inserting articles into the database
insert_query_template = f"""
INSERT IGNORE INTO {db_name}.{table_name}
(source, author, title, description, url, urlToImage, publishedAt, content, full_text)
VALUES (:source, :author, :title, :description, :url, :urlToImage, :publishedAt, :content, :full_text)
"""

with engine.connect() as connection:
    for article in tesla_articlse:
        # Extracting necessary details from the 'source' field
        source_name = article['source']['name'] if article['source'] else None

        # Preparing data for insertion
        data = {
            "source": source_name,
            "author": article.get('author'),
            "title": article.get('title'),
            "description" : article.get('description'),
            "url": article.get('url'),
            "urlToImage": article.get('urlToImage'),
            "publishedAt": article.get('publishedAt'),
            "content": article.get('content'),
            "full_text": article.get('full_text')
        }

        # Execute the insertion
        connection.execute(text(insert_query_template), data)
    # Commit the transaction after all insertions
    connection.commit()


In [40]:
# Retrieving articles from the database
retrieve_articles_query = text(f"SELECT * FROM {db_name}.{table_name}")
articles = []

with engine.connect() as connection:
    result_proxy = connection.execute(retrieve_articles_query)
    # Convert column names to a list
    column_names = list(result_proxy.keys())
    results = result_proxy.fetchall()  # Fetch all results
    for row in results:
        article = {column_names[i]: value for i, value in enumerate(row)}
        articles.append(article)


In [87]:
import openai

# Initialize OpenAI API
openai.api_key = ''

def get_sentiment(text):
    prompt = f"Given the following article text, determine its sentiment on a scale from -1 (very negative) to 1 (very positive): {text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    sentiment_value = response.choices[0].message['content'].strip()

    # Convert sentiment_value to float if possible
    try:
        return float(sentiment_value)
    except ValueError:
        return 0  # Return neutral if there's an issue

def get_entities(text):
    prompt = f"List all entities in this article: {text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    entities = response.choices[0].message['content'].strip().split(', ')
    return entities

def get_feedback(text):
    prompt = f"Give a brief feedback about this article: {text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    feedback = response.choices[0].message['content'].strip().split(', ')
    return feedback

def truncate_text(text, max_chars=4000):
    """Truncate text to a specified number of characters."""
    return text[:max_chars]

# Analyze each article
for article in articles:
    truncated_text = truncate_text(article['full_text'])
    article['sentiment'] = get_sentiment(truncated_text)
    article['entities'] = get_entities(truncated_text)
    article['feedback'] = get_feedback(truncated_text)



In [88]:
from sqlalchemy import text

# Create table for sentiment
create_sentiment_table_query = text(f"""
CREATE TABLE IF NOT EXISTS {db_name}.poh2005_sentiment (
    url VARCHAR(255),
    sentiment_score VARCHAR(255)
)
""")

# Create table for entities
create_entities_table_query = text(f"""
CREATE TABLE IF NOT EXISTS {db_name}.poh2005_entities (
    url VARCHAR(255),
    entity VARCHAR(255)
)
""")

# Create table for feedback
create_feedback_table_query = text(f"""
CREATE TABLE IF NOT EXISTS {db_name}.poh2005_feedback (
    url VARCHAR(255),
    feedback VARCHAR(255)
)
""")

with engine.connect() as connection:
    connection.execute(create_sentiment_table_query)
    connection.execute(create_entities_table_query)
    connection.execute(create_feedback_table_query)


In [89]:
# Insert query templates
sentiment_query_template = f"""
INSERT IGNORE INTO
public.poh2005_sentiment(url, sentiment_score)
VALUES (:url, :sentiment_score)
"""

entities_query_template = f"""
INSERT IGNORE INTO
public.poh2005_entities(url, entity)
VALUES (:url, :entity)
"""

feedback_query_template = f"""
INSERT IGNORE INTO
public.poh2005_feedback(url, feedback)
VALUES (:url, :feedback)
"""

with engine.connect() as connection:
    for article in articles:
        url = article["url"]
        sentiment = article["sentiment"]
        entities = article["entities"]
        feedback = article["feedback"]

        # Insert sentiment
        try:
            print(f"Inserting sentiment for article {article['title']}, URL: {url}")
            connection.execute(text(sentiment_query_template), {"url": url, "sentiment_score": sentiment})
        except Exception as e:
            print(f"Error inserting sentiment for article {article['title']}, URL: {url}. Error: {str(e)}")

        # Insert entities
        for entity in entities:
            try:
                print(f"Inserting entity '{entity}' for article {article['title']}, URL: {url}")
                connection.execute(text(entities_query_template), {"url": url, "entity": entity})
            except Exception as e:
                print(f"Error inserting entity '{entity}' for article {article['title']}, URL: {url}. Error: {str(e)}")

         # Insert sentiment
        try:
            print(f"Inserting feedback for article {article['title']}, URL: {url}")
            connection.execute(text(feedback_query_template), {"url": url, "feedback": feedback})
        except Exception as e:
            print(f"Error inserting feedback for article {article['title']}, URL: {url}. Error: {str(e)}")

    connection.commit()


Streaming output truncated to the last 5000 lines.
3. Genesis GV60
4. Tesla' for article I compared Tesla rivals from Kia, Hyundai, and Genesis — and found the decision boils down to if you prefer style, luxury, or performance, URL: https://www.businessinsider.com/kia-ev6-hyundai-ioniq-5-genesis-gv60-tesla-rival-ev-2023-9
Inserting feedback for article I compared Tesla rivals from Kia, Hyundai, and Genesis — and found the decision boils down to if you prefer style, luxury, or performance, URL: https://www.businessinsider.com/kia-ev6-hyundai-ioniq-5-genesis-gv60-tesla-rival-ev-2023-9
Error inserting feedback for article I compared Tesla rivals from Kia, Hyundai, and Genesis — and found the decision boils down to if you prefer style, luxury, or performance, URL: https://www.businessinsider.com/kia-ev6-hyundai-ioniq-5-genesis-gv60-tesla-rival-ev-2023-9. Error: (pymysql.err.OperationalError) (1241, 'Operand should contain 1 column(s)')
[SQL: 
INSERT IGNORE INTO
public.poh2005_feedback(url,